# Import the Necessary Libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchsummary import summary # pip install torch-summary
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Download the CIFAR 10 Dataset

In [4]:
# Get the training data and create a transform to ensure the data is made up of tensors
# why are we not normalizing the data between 0 and 1?
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform=transform)

# load the training data and transform it
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

# Get the test data
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform=transform)

# Now load the test data with a batch size of 32: Do we need to shuffle the test data?
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:01<00:00, 89881892.66it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Building the CNN Model

In [13]:
import math

# Calculate the output size of a convolutional layer with "same" padding
def calculate_output_size(input_size, kernel_size, stride, padding):
    return math.floor((input_size + 2 * padding - kernel_size) / stride) + 1

 # write your code below and ensure your padding is correct and well calculated
# calculating the padding
def calculate_padding(input_size, kernel_size, stride):
    padding = ((input_size - 1) * stride - input_size + kernel_size) // 2
    return math.floor(padding)

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # Create instance variables for the layers you will pass in the forward method


        # padding_number = 1
        padding_number1 = calculate_padding(32, 4, 1)
        self.convLayer1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=1, padding=padding_number1)
        self.bn1 = torch.nn.BatchNorm2d(32)

        padding_number2 = calculate_padding(32, 3, 2)
        self.convLayer2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=padding_number2)
        self.bn2 = torch.nn.BatchNorm2d(32)

        padding_number3 = calculate_padding(32, 3, 1)
        self.convLayer3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=padding_number3)
        self.bn3 = torch.nn.BatchNorm2d(64)

        padding_number4 = calculate_padding(32, 3, 2)
        self.convLayer4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=padding_number4)
        self.bn4 = torch.nn.BatchNorm2d(64)

        # Calculate the output size of the last convolutional layer
        last_conv_output_size = calculate_output_size(32, 4, 1, padding_number1)  # Assuming input size 32x32
        last_conv_output_size = calculate_output_size(last_conv_output_size, 3, 2, padding_number2)
        last_conv_output_size = calculate_output_size(last_conv_output_size, 3, 1, padding_number3)
        last_conv_output_size = calculate_output_size(last_conv_output_size, 3, 2, padding_number4)


        self.fclayer1 = torch.nn.Linear(64 * last_conv_output_size * last_conv_output_size, 128)
        self.bn5 = torch.nn.BatchNorm1d(128)

        self.fclayer2 = torch.nn.Linear(128 , 10)


    def forward(self, x):
        # complete your forward method
        x=torch.nn.functional.leaky_relu(self.bn1(self.convLayer1(x)))
        x=torch.nn.functional.leaky_relu(self.bn2(self.convLayer2(x)))

        x=torch.nn.functional.leaky_relu(self.bn3(self.convLayer3(x)))
        x=torch.nn.functional.leaky_relu(self.bn4(self.convLayer4(x)))

        x = x.view(x.size(0), -1)

        x=torch.nn.functional.leaky_relu(self.bn5(self.fclayer1(x)))

        x=torch.nn.functional.dropout(x,p=.5)

        x=self.fclayer2(x)

        return torch.nn.functional.log_softmax(x, dim=-1)

cnn_model = CNN().to(device)
summary(cnn_model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 31, 31]           1,568
       BatchNorm2d-2           [-1, 32, 31, 31]              64
            Conv2d-3           [-1, 32, 31, 31]           9,248
       BatchNorm2d-4           [-1, 32, 31, 31]              64
            Conv2d-5           [-1, 64, 31, 31]          18,496
       BatchNorm2d-6           [-1, 64, 31, 31]             128
            Conv2d-7           [-1, 64, 31, 31]          36,928
       BatchNorm2d-8           [-1, 64, 31, 31]             128
            Linear-9                  [-1, 128]       7,872,640
      BatchNorm1d-10                  [-1, 128]             256
           Linear-11                   [-1, 10]           1,290
Total params: 7,940,810
Trainable params: 7,940,810
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forw

# Set the Loss Function and Optimizer

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.0005)

# Write the Train Function

In [8]:
def train(dataloader, model, criterion, optimizer):
    model.train()
    batch_bar = tqdm(total=len(dataloader), position=0, leave=False, dynamic_ncols=True, desc='Train')
    total_loss = 0
    total_accuracy = 0

    for i, (imgs, labels) in enumerate(dataloader):
        # write your code below for the train function
        # Move data to device which is cpu
        imgs, labels = imgs.to(device), labels.to(device)

        # Forward pass
        outputs = model(imgs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).sum().item()

        total_loss += loss.item() * imgs.size(0)
        total_accuracy += accuracy
        # total_samples += labels.size(0)

        # update the progress bar
        batch_bar.set_postfix(loss=loss.item(), accuracy=100 * accuracy / len(labels))
        batch_bar.update()
    batch_bar.close()
    return total_loss/len(dataloader), 100*total_accuracy/(len(dataloader)*32)

# Write the Evaluate Function to get the Test Accuracy

In [9]:
def evaluate(dataloader, model):
    model.eval()
    # write your code for the evaluate function
    accuracy = 0
    total = 0

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)

            # Get predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Total number of samples
            total += labels.size(0)

            # Number of correctly classified samples
            accuracy += (predicted == labels).sum().item()

    # Calculate test accuracy
    test_accuracy = 100 * accuracy / total
    return test_accuracy


# Train the CNN Model

In [10]:
for epoch in range(0, 10):
    train_loss, train_acc = train(train_dataloader, cnn_model, criterion, optimizer)
    print(f"Epoch {epoch+1}/{10} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_acc:.4f}")

Epoch 1/10 - Train Loss: 43.8190 - Train Accuracy: 51.1896


Epoch 2/10 - Train Loss: 32.8833 - Train Accuracy: 63.8176


Epoch 3/10 - Train Loss: 28.4931 - Train Accuracy: 68.8680


Epoch 4/10 - Train Loss: 25.5283 - Train Accuracy: 71.9630


Epoch 5/10 - Train Loss: 23.0007 - Train Accuracy: 74.9420


Epoch 6/10 - Train Loss: 20.8042 - Train Accuracy: 77.4172


Epoch 7/10 - Train Loss: 18.7343 - Train Accuracy: 79.4706


Epoch 8/10 - Train Loss: 17.0749 - Train Accuracy: 81.3980


Epoch 9/10 - Train Loss: 15.5405 - Train Accuracy: 82.8835


Epoch 10/10 - Train Loss: 14.1802 - Train Accuracy: 84.2710


# Get the Test Accuracy

In [11]:
test_acc = evaluate(test_dataloader, cnn_model)
print("The test accuracy is: ",test_acc)


The test accuracy is:  72.04
